In [1]:
import pandas as pd
from datetime import datetime
import datetime

printingIndex=27;
speakingStr = "./dfs/{}/speaking_annotations.csv".format(printingIndex);
motionStr = "./action/{}/motion_log.csv".format(printingIndex);
speaking = pd.read_csv(speakingStr) 
motion = pd.read_csv(motionStr) 

text =  motion.iloc[:, 1] #this extracts all of the talking text
s = pd.Series(text)
firstInstanceIndex = s.where(s=='speech').last_valid_index()
if(firstInstanceIndex == None):
    firstInstanceIndex = 0
    
index = motion.index
number_of_rows = len(index)
difference = firstInstanceIndex - number_of_rows + 1

temp = motion[difference:]
temp = temp.reset_index(drop=True)
temp.columns = ["index", "comments", "question difficulty", "participant", "number", "time"]

for index, row in temp.iterrows():
    rowVar = row["time"]
    hour = int(rowVar[11:13])
    minute = int(rowVar[14:16])
    second = int(rowVar[17:19])
    microsecond = int(rowVar[20:22]) * 10000
    currDate = datetime.datetime(2000, 1, 1, hour, minute, second, microsecond)
    temp.at[index, 'time'] = currDate;
    
temp["left"] = 0;
temp["center"] = 0;
temp["right"] = 0;
speaking["speechStart"] = None;
speaking["speechEnd"] = None;
firstNumericTime = temp.iloc[0]["time"]

prev = 0;
curr = 0;
for index, row1 in speaking.iterrows():
    rowVar = row1["time"];
    currRight = row1["right"];
    currLeft = row1["left"];
    currCenter = row1["center"];
    currBot = row1["bot"];
    if(currRight == 1 or currLeft ==1 or currCenter==1 or currBot==1):
        curr = 1;
    else:
        curr = 0;
    #print rowVar
    hour = int(rowVar[0:2])
    minute = int(rowVar[3:5])
    second = int(rowVar[6:8])
    microsecond = int(rowVar[9:11]) * 10000
    currDate = datetime.datetime(2000, 1, 1, hour, minute, second, microsecond)
    #print currDate
    speaking.at[index, 'time'] = currDate
    
    if(prev == 0 and curr == 1):
        prev = 1;
        speaking.at[index, 'speechStart'] = 1;
    elif(prev == 1 and curr ==0):
        prev = 0;
        speaking.at[index, 'speechEnd'] = 1;
        
speaking["comments"] = 0;
speaking["question difficulty"] = 0;
speaking["participant"] = 0;
speaking["number"] = 0;

firstNumericTime = speaking.iloc[0]["time"]

import numpy as np;
speakingLength = len(speaking.index);
motionLength = len(temp.index);
i = 0; #motion
j = 0; #speak
store = [];

while(1):
    motionTime = temp.iloc[i]["time"];
    speakingTime = speaking.iloc[j]["time"];
    if(motionTime > speakingTime):
        standardTime = speakingTime.minute*60000000  + speakingTime.second * 1000000 + speakingTime.microsecond;
        standardTime = standardTime / 10000;
        store.append([speaking.iloc[j]["left"], speaking.iloc[j]["center"], speaking.iloc[j]["right"], 
                      speaking.iloc[j]["bot"], speaking.iloc[j]["speechStart"], speaking.iloc[j]["speechEnd"], 
                      standardTime, None, None, 0]);
        j = j + 1;
    else:
        standardTime = motionTime.minute*60000000  + motionTime.second * 1000000 + motionTime.microsecond;
        standardTime = standardTime / 10000;
        participant = temp.iloc[i]["participant"];
        difficulty = temp.iloc[i]["question difficulty"];
        if(difficulty == np.nan or difficulty == "" or difficulty == None):
            difficulty = None;
        right = 0;
        left = 0;
        center = 0;
        if(participant == "Participant 3"):
            center = 1;
        elif(participant == "Participant 1"):
            right = 1;
        elif(participant == "Participant 5"):
            left = 1;
        store.append([left, center, right, 0, None, None, standardTime, 
                      temp.iloc[i]["comments"], difficulty, temp.iloc[i]["number"]]);
        i = i + 1;
    if(i == motionLength or j == speakingLength):
        break;
        
if(i != motionLength):
    while(1):
        standardTime = motionTime.minute*60000000  + motionTime.second * 1000000 + motionTime.microsecond;
        standardTime = standardTime / 10000;
        participant = temp.iloc[i]["participant"];
        difficulty = temp.iloc[i]["question difficulty"];
        if(difficulty == np.nan or difficulty == ""):
            difficulty =None;
        right = 0;
        left = 0;
        center = 0;
        if(participant == "Participant 3"):
            center = 1;
        elif(participant == "Participant 1"):
            right = 1;
        elif(participant == "Participant 5"):
            left = 1;
        store.append([left, center, right, 0, None, None, standardTime, temp.iloc[i]["comments"], difficulty, 
                      temp.iloc[i]["number"]]);
        i = i + 1;
        if(i == motionLength):
            break;

if(j != speakingLength):
    while(1):
        speakingTime = speaking.iloc[j]["time"];
        standardTime = speakingTime.minute*60000000  + speakingTime.second * 1000000 + speakingTime.microsecond;
        standardTime = standardTime / 10000;
        store.append([speaking.iloc[j]["left"], speaking.iloc[j]["center"], speaking.iloc[j]["right"], 
                      speaking.iloc[j]["bot"], speaking.iloc[j]["speechStart"], speaking.iloc[j]["speechEnd"], 
                      standardTime, None, None, 0]);
        j = j + 1;
        if(j == speakingLength):
            break;
            
from pandas import DataFrame
labels=["left", "center", "right", "bot", "speechStart", "speechEnd", "time", "comments", "question difficulty", "duration"];
df = DataFrame(store,columns=labels);

df["trialNum"] = printingIndex; #change each time
csvStr = "./outputNew/out{}.csv".format(printingIndex);
df.to_csv(csvStr, index=False)